In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.linalg as la
import itertools

# Algoritmos Genéticos

Algoritmos genéticos são classes de heurísticas que servem para resolver problemas computacionais aparentemente sem solução ou com tempo de resolução de complexidade exponencial, as vezes o tempo de resolução pode durar até bilhões de anos. Esses algoritmos se utilizam de técnicas baseadas na metáfora da evolução genética por seleção natural proposta por Charles Darwin.

Assim, temos um algoritmo que possui uma certa **população inicial** de indivíduos, inicializada de forma totalmente aleatória (estocástica ou probabilística) cada **indivíduo** possui um **cromossomo** que nada mais é do que uma **sequência de genes** em uma **bit string**. Esses genes são os **parâmetros**, que consitutem o fenótipo, a serem analisados do problema convertidos em bit string. A cada iteração, serão realizados sobre a população algumas operações evolutivas:

- Seleção
- Reprodução (Crossing Over e recombinação genética)
- Mutação

A **seleção** se faz com uma **função de aptidão**, na qual realiza cálculos e rotula quais são os indivíduos da população mais bem adaptados. Assim, a qualidade de um indivíduo é calculada de acordo com a sua adaptação ao meio, feita pela função de aptidão.

Após a seleção e rotulação dos indivíduos mais bem adaptados ao meio, chegamos à fase de reprodução. Nessa fase, totalmente estocástica, faremos a troca dos bits (genes) dos individuos mais bem adaptados ao meio, aumentando a variabilidade genética da população e fazendo com que indivíduos mais bem adaptados que seus pais surjam nas próximas iterações. Os indivíduos menos adaptados não devem ser descartados, pois assim reduziríamos a variabilidade genética da população e provavelmente o algoritmo iria convergir em um máximo local e não global da função de aptidão.

A **mutação** deve ocorrer com uma probabilidade bem baixa, mudando apenas um bit de um dos indivíduos na reprodução. Esse processo aumenta a taxa de convergência se for feito com probabilidade baixa de ocorrência, impedindo que a população fique estagnada em um máximo local da função de aptidão. No entanto, se for feita muito constantemente, pode aumentar e muito o tempo de convergência do algoritmo ou até mesmo não convergir.

## Busca Genética por Circuitos Lógicos

O vetor **OutPAM** precisa ser comparado à matriz **Out**. O OutPAM tem a
saída que nós queremos, o vetor OR. Ele é um vetor coluna. Por outro lado, a
matriz Out tem 3 vetores colunas, e em uma dessas colunas, queremos encontrar
o vetor **SPAM** variando os parâmetros α, β e γ. Mas para isso, precisamos de
um método de comparacão de vetores, e aqui usaremos um truque.

## A função de aptidão e o truque de kernel

Sem perda de generalidade, considere que a primeira coluna do vetor $Out (Out_{1})$ contém valores próximos da saída que queremos encontrar. Assim, para verificar isso basta fazermos a diferença $OutPAM - Out_{1}$. Quando, em uma situação perfeita, tivermos esses dois vetores iguais, teremos assim um vetor resultante nulo. Nosso objetivo é chegar o mais próximo possível desse resultado. 

Logo, devemos variar os parâmetros $\alpha, \beta, \gamma, \epsilon$ que produzem o vetor $Out_{1}$ de forma a deixá-lo o mais próximo possível de $OutPAM$. Uma forma de mesurar isso é com a seguinte $função de aptidão$:

$$f(\alpha, \beta, \gamma, \epsilon) = e^{-|OutPAM - Out_{1}(\alpha, \beta, \gamma, \epsilon)|}$$

Logo, o algoritmo varia os parâmetros $\alpha, \beta, \gamma$ a fim de máximizar a **função de aptidão** acima, que ocorre quando temos $OutPAM$ muito próximo de $Out_{1}$. Cada quadra ($\alpha, \beta, \gamma, \epsilon$) é chamada de indivíduo da população de possíveis soluções (outras quadras) do problema.

In [2]:
def ArranjoTriangular(alpha, beta, gama):
    """
    Retorna a matriz triangular A com parâmetros alpha, beta e gamma.
    complex(x, y) retorna um número complexo da forma x + yi, onde i é o número imaginário
    
    Parâmetros
    --------------------------------------
    :params alpha, beta, gama: alpha é um número real
    
    Retorno
    --------------------------------------
    :return: retorna a matriz arranjo triangular como numpy array cujos valores estão de acordo com os parâmetros recebidos
    """
    A = np.array([[0, complex(0, gama), complex(0, beta)], 
                  [complex(0, gama), 0, complex(0, alpha)], 
                  [complex(0, beta), complex(0, alpha), 0]])
    
    return A

In [3]:
def MatrizTransferenciaTriangular(A):
    """
    Retorna a matriz de transferência triangular T a partir de uma matriz de arranjo triangular A a partir dos passos já citados
    Erro percentual de 10^(-14) a 10^(-16), ínfimo.
    
    Parâmetros
    --------------------------------------
    :param A: matriz de Arranjo Triangular como numpy array
    
    Retorno
    
    :return:  matriz numpy array de transferência triangular T calculada com erro ínfimo.
    """
    eigvals, eigvecs = la.eig(A)
    D = np.array([[eigvals[0], 0, 0], 
                  [0, eigvals[1], 0], 
                  [0, 0, eigvals[2]]])
    P = eigvecs.copy()
    E = np.array([[np.exp(eigvals[0]), 0, 0], 
                  [0, np.exp(eigvals[1]), 0], 
                  [0, 0, np.exp(eigvals[2])]])
    
    if np.allclose(A, P @ D @ la.inv(P), atol=1e-17):
        raise ValueError(f'Parameter Matrix A has not property A == PDP^(-1) for a tolerance of {1e-17}')
    
    # T é a matriz de transferência
    T = P @ E @ la.inv(P)
    
    return T

In [4]:
def Tv(NumberOfInputs):
    """
    Retorna uma tabela verdade (TruthTable) dado um circuito com n inputs maiores ou iguais a 1
    Checa se a variável NumberOfInputs é inteira. Se sim, continua normalmente, se não lança uma exceção com a mensagem
    'Parameter \'NumberOfInputs\' must be a integer'

    Parâmetros
    ------------------------------------------------
    :param NumberOfInputs: número de inputs do circuito lógico, deve ser um número inteiro (obrigatoriamente)
    
    Retorno
    ------------------------------------------------
    :return: Retorna a tabela verdade como uma lista do Python (caso necessário, transformar o retorno em numpy array)
    """
    if not isinstance(NumberOfInputs, int):
        raise ValueError('Parameter \'NumerOfInputs\' must be a integer')
    if NumberOfInputs < 1:                                                   
        raise ValueError('Parameter \'NumberOfInputs\' must be greater or equal to 1')
    else:
        table = list(itertools.product ([0, 1], repeat = NumberOfInputs))    
        table = np.array(table, dtype=float)
        return table

In [5]:
def PAM(TruthTable, epsilon):
    """
    Função que retorna uma modulação PAM para uma dada tabela verdade (TruthTable) e certo epsilon real arbitrário.
    Não modifica a tabela verdade original, apenas realiza alterações em uma cópia e a retorna no final.
    Funciona para qualquer tabela verdade (TruthTable) com n inputs (Matriz não precisa ser quadrada)

    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada e transformada em modulação PAM
    :param epsilon: epsilon no qual os valores serão calculados
    
    Retorno
    ------------------------------------------------
    :return: retorna um numpy array como matriz da modulação PAM da tabela verdade recebida como parâmetro
    """
    if epsilon <= 0 or epsilon >= 1:
        raise ValueError(f'Epsilon must be major than 0 and minor than 1')
    PamTable = TruthTable.copy() # Como utiliza uma cópia, PamTable será uma matriz com elementos do tipo float
    rows, columns = PamTable.shape
    for row in range(rows):
        for column in range(columns):
            if PamTable[row][column] == 1:
                PamTable[row][column] = 1 + epsilon
            elif PamTable[row][column] == 0:
                PamTable[row][column] = 1 - epsilon
            else:
                raise ValueError('Truth Table must be filled just with zeros and ones')
    return PamTable

In [6]:
def InvPAM(PAMTable):
    """
    Função que retorna uma nova tabela verdade com modulação bniária (processo inverso da modulação PAM)
    
    Parâmetros
    ------------------------------
    :param PAMTable: Tabela verdade em modulação PAM (1 +- epsilon)
    
    Retorno
    ------------------------------
    :return: Retorna uma nova matriz como numpy array que representa a tabela verdade original (inversa da PAM)
    """
    rows, columns = PAMTable.shape
    ResultTable = np.ones(shape=(rows, columns), dtype=float)
    for row in range(rows):
        for column in range(columns):
            if PAMTable[row][column] < 1:
                ResultTable[row][column] = 0
    return ResultTable

In [7]:
def OrTruthTableGenerator(TruthTable):
    """
    Retorna uma tabela verdade do circuito lógico OR de acordo com uma tabela verdade (TruthTable). 
    Checa se a tabela está vazia, se sim a retorna (vazia), se não, realiza os cálculos.
    
    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada
    
    Retorno
    ------------------------------------------------
    :return: Retorna a coluna (numpy array) com os valores da tabela verdade OR dos inputs da tabela verdade recebida 
    como parâmetro
    """
    rows, columns = TruthTable.shape
    ResultTable = np.zeros(shape=(rows, 1), dtype=float)
    
    if TruthTable.size == 0:
        return TruthTable
    
    for row in range(rows):
        BooleanRowArray = TruthTable[row][:]
        if np.any(BooleanRowArray):
            ResultTable[row][0] = 1
        
    return ResultTable

In [8]:
def AndTruthTableGenerator(TruthTable):
    """
    Retorna uma tabela verdade do circuito lógico AND de acordo com uma tabela verdade (TruthTable). 
    Checa se a tabela está vazia, se sim a retorna (vazia), se não, realiza os cálculos.
    
    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada
    
    Retorno
    ------------------------------------------------
    :return: Retorna a coluna (numpy array) com os valores da tabela verdade AND dos inputs da tabela verdade recebida 
    como parâmetro
    """
    rows, columns = TruthTable.shape
    ResultTable = np.zeros(shape=(rows, 1), dtype=float)
    
    if TruthTable.size == 0:
        return TruthTable
    
    for row in range(rows):
        BooleanRowArray = TruthTable[row][:]
        if np.all(BooleanRowArray):
            ResultTable[row][0] = 1;
    
    return ResultTable

In [9]:
OR = OrTruthTableGenerator(Tv(3))
A = ArranjoTriangular(1.2, 2.3, 3.4)
T = MatrizTransferenciaTriangular(A)
Inp = PAM(Tv(3), 0.5)

Out = T @ Inp.T
Out = Out.T
OutPAM = PAM(OR, 0.5)

In [10]:
print(f'A: \n{A}\n\nT: \n{T}\n\nInp: \n{Inp}\n\nOut: \n{Out}\n\nOutPAM: \n{OutPAM}')

A: 
[[0.+0.j  0.+3.4j 0.+2.3j]
 [0.+3.4j 0.+0.j  0.+1.2j]
 [0.+2.3j 0.+1.2j 0.+0.j ]]

T: 
[[-0.48247329-0.18796246j  0.43582066-0.65595554j  0.13308381-0.30653062j]
 [ 0.43582066-0.65595554j -0.20109041-0.42666534j -0.34652227+0.1929361j ]
 [ 0.13308381-0.30653062j -0.34652227+0.1929361j   0.25716171-0.81541003j]]

Inp: 
[[0.5 0.5 0.5]
 [0.5 0.5 1.5]
 [0.5 1.5 0.5]
 [0.5 1.5 1.5]
 [1.5 0.5 0.5]
 [1.5 0.5 1.5]
 [1.5 1.5 0.5]
 [1.5 1.5 1.5]]

Out: 
[[ 0.04321559-0.57522431j -0.05589601-0.44484239j  0.02186163-0.46450227j]
 [ 0.1762994 -0.88175493j -0.40241828-0.25190628j  0.27902333-1.2799123j ]
 [ 0.47903624-1.23117985j -0.25698643-0.87150773j -0.32466064-0.27156617j]
 [ 0.61212006-1.53771047j -0.6035087 -0.67857162j -0.06749894-1.08697619j]
 [-0.43925771-0.76318677j  0.37992464-1.10079792j  0.15494544-0.77103289j]
 [-0.30617389-1.0697174j   0.03340237-0.90786182j  0.41210715-1.58644292j]
 [-0.00343705-1.41914231j  0.17883423-1.52746326j -0.19157683-0.57809679j]
 [ 0.12964676-1.7256729

## Erro de Broadcasting (Numpy)

Para conseguir o vetor coluna **Out1**, não basta apenas fazer ``Out1 = Out[:, 0]``, pois assim teremos apenas uma lista com os valores da primeira coluna da matriz **Out**. Logo, precisamos criar um numpy array baseado nesses valores:

``
Out1 = np.array([Out[:, 0]])
``

Entretanto, ainda temos um erro, pois Out1 será um vetor linha e não coluna, implicando assim em um erro de **Broadcasting** do numpy ao realizar a operação $OutPAM - Out_{1}$, pois iríamos realizar a subtração entre uma matriz de dimensões (m, 1) com outra de dimensões (1, m).

Assim, precisamos além de criar o numpy array, mudar a referência de $Out_{1}$ para a sua transposta:

``
Out1 = np.array([Out[:, 0]])
Out1 = Out1.T
``

## Construção do GA (Genetic Algorithm):

In [15]:
def AptidaoArranjoTriangular(alpha, beta, gamma, epsilon, target):
    A = ArranjoTriangular(alpha, beta, gamma)
    T = MatrizTransferenciaTriangular(A)
    Inp = PAM(Tv(3), epsilon)
    Out = T @ Inp.T
    Out = Out.T
    OutPAM = PAM(target, epsilon)
    
    Out1 = np.array([Out[:,0]])
    Out1 = Out1.T
    kernel = math.e**(-np.linalg.norm(OutPAM - Out1))
    
    return kernel

In [19]:
kernel = AptidaoArranjoTriangular(3.4, 4.3, 5.4, 0.35, OR)
print(f'Kernel: \n\n{kernel}')

Kernel: 

0.0016204814247774194
